# MLOps Data Preparation
## Credit Risk Classification — Analysis of the South German Credit Dataset




#Libraries

In [1]:
# =============================================================
# Import-capture robusto + listado SOLO de lo que importas tú
# =============================================================
import builtins, sys
from importlib import import_module
from importlib.metadata import packages_distributions, version, PackageNotFoundError

_CAPTURE_ACTIVE = False
_CAPTURED = set()
_ORIG_IMPORT = builtins.__import__

# alias útiles módulo -> distribución PyPI
_ALIAS = {
    "sklearn": "scikit-learn",
    "PIL": "Pillow",
}

# Ruido típico de IPython/Jupyter que no queremos
_IGNORE = {
    "IPython","matplotlib_inline","jupyter","jupyter_core","jupyter_client",
    "ipykernel","traitlets","prompt_toolkit","pygments","pexpect","pickleshare",
    "debugpy","parso","wcwidth","ptyprocess","backcall","decorator"
}

def _import_hook(name, globals=None, locals=None, fromlist=(), level=0):
    if _CAPTURE_ACTIVE:
        top = name.split('.')[0]
        # evita stdlib y ruido
        if hasattr(sys, "stdlib_module_names"):
            is_std = top in sys.stdlib_module_names
        else:
            # fallback muy simple para entornos sin stdlib_module_names
            is_std = top in {"sys", "os", "json", "re", "math", "time", "itertools", "functools"}
        if (not is_std) and (top not in _IGNORE):
            _CAPTURED.add(top)
    return _ORIG_IMPORT(name, globals, locals, fromlist, level)

def start_import_capture(reset=True):
    """Activa el hook de importación. Usa reset=True para limpiar capturas previas."""
    global _CAPTURE_ACTIVE, _CAPTURED
    if reset:
        _CAPTURED = set()
    builtins.__import__ = _import_hook
    _CAPTURE_ACTIVE = True
    print("✅ Import capture STARTED")

def stop_import_capture(show_requirements=True, include_deps=False):
    """Desactiva el hook y opcionalmente imprime requirements.
       include_deps=False -> solo los módulos que TÚ importaste explícitamente.
       include_deps=True  -> intenta resolver distribuciones PyPI (puede añadir dependencias)."""
    global _CAPTURE_ACTIVE
    builtins.__import__ = _ORIG_IMPORT
    _CAPTURE_ACTIVE = False
    print("🛑 Import capture STOPPED")
    if show_requirements:
        print("\n============================================")
        print("🔹 IMPORTED PACKAGES (requirements format)")
        print("============================================")
        _print_requirements(include_deps=include_deps)

def _safe_print_line(name, ver):
    if name and str(name).strip() and ver and str(ver).strip():
        print(f"{name}=={ver}")

def _print_requirements(include_deps=False):
    """Si include_deps=False: intenta __version__ del módulo (solo lo que importaste).
       Si include_deps=True: resuelve distribución con importlib.metadata (más ruidoso)."""
    mapping = packages_distributions()

    seen = set()
    for mod in sorted(_CAPTURED):
        if mod in seen:
            continue

        # 1) Solo lo importado explícito (recomendado: include_deps=False)
        if not include_deps:
            try:
                m = import_module(mod)
                ver = getattr(m, "__version__", None)
                # algunos modulos usan nombres distintos para la distribución
                dist_name = _ALIAS.get(mod, mod)
                if ver:
                    _safe_print_line(dist_name, ver)
                    seen.add(mod)
                # si no tiene __version__, lo omitimos (evita ValueError)
            except Exception:
                pass
            continue

        # 2) Modo dependencias: resolver distribución PyPI
        dist_names = mapping.get(mod, [])
        dist = _ALIAS.get(mod) or (dist_names[0] if dist_names else mod)

        # evitar nombres vacíos que causan ValueError
        if not dist or not str(dist).strip():
            continue

        try:
            ver = version(dist)
            _safe_print_line(dist, ver)
            seen.add(mod)
        except PackageNotFoundError:
            # intento final con el nombre del módulo
            try:
                ver = version(mod)
                _safe_print_line(mod, ver)
                seen.add(mod)
            except PackageNotFoundError:
                pass


In [2]:
start_import_capture()


✅ Import capture STARTED


In [3]:
# CARGAMOS LAS LIBRERIAS
!pip install category_encoders

# --- Google Colab específicos ---
from google.colab import userdata
from google.colab import drive

# --- Manejo de datos ---
import pandas as pd
import numpy as np

# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.7 MB/s eta 0:00:00


#Dataset Preview

In [4]:
#IMPORTAMOS EL DATASET
drive.mount('/content/drive', force_remount=True)
FOLDER_ID = userdata.get('FOLDER_ID')
TARGET = f"/content/drive/.shortcut-targets-by-id/{FOLDER_ID}"
import os
os.chdir(TARGET)

Mounted at /content/drive


In [5]:
#AGREGAR IMPORTACION DE DATA

In [6]:
#VISUALIZAMOS EL DATASET
#Preview Dataset
df = pd.read_parquet('trabajo_grupal_mlops/data/01_df_data_preparation_01.parquet')
df.T

,0,1,2,3,4,5,6,7,8,9,...,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019
laufkont,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,...,2.0,4.0,2.0,2.0,1.0,2.0,2.0,4.0,3.0,4.0
laufzeit,18.0,9.0,12.0,12.0,12.0,10.0,8.0,6.0,18.0,24.0,...,39.0,15.0,None,30.0,9.0,42.0,36.0,12.0,18.0,36.0
moral,4.0,4.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,...,3.0,4.0,2.0,3.0,2.0,1.0,2.0,3.0,2.0,2.0
verw,2.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.0,null,...,6.0,2.0,3.0,3.0,3.0,1.0,0.0,5.0,2.0,2.0
hoehe,1049.0,2799.0,841.0,2122.0,2171.0,2241.0,3398.0,1361.0,1098.0,3758.0,...,11760.0,1520.0,766.0,1919.0,1364.0,9283.0,14318.0,1555.0,3049.0,10974.0
sparkont,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,2.0,5.0,3.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0
beszeit,2.0,3.0,4.0,3.0,3.0,2.0,4.0,2.0,1.0,1.0,...,4.0,5.0,3.0,2.0,4.0,1.0,5.0,5.0,2.0,1.0
rate,4.0,2.0,2.0,3.0,4.0,1.0,1.0,2.0,4.0,1.0,...,2.0,4.0,4.0,4.0,3.0,1.0,4.0,4.0,1.0,4.0
famges,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,...,3.0,3.0,3.0,3.0,3.0,invalid,3.0,3.0,2.0,2.0
buerge,1.0,1.0,1.0,1.0,None,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


#Data Analysis & Cleaning


In [7]:
#RENOMBRAMOS LAS COLUMNAS
#Rename Columns headers
col_ing = ['status', 'duration', 'credit_history', 'purpose', 'amount',
           'savings', 'employment_duration', 'installment_rate', 'personal_status_sex',
           'other_debtors', 'present_residence', 'property', 'age', 'other_installment_plans',
           'housing', 'number_credits', 'job', 'people_liable', 'telephone', 'foreign_worker',
           'credit_risk','mixed']

df.columns = col_ing
df.head(10).T


,0,1,2,3,4,5,6,7,8,9
status,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0
duration,18.0,9.0,12.0,12.0,12.0,10.0,8.0,6.0,18.0,24.0
credit_history,4.0,4.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0
purpose,2.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.0,null
amount,1049.0,2799.0,841.0,2122.0,2171.0,2241.0,3398.0,1361.0,1098.0,3758.0
savings,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
employment_duration,2.0,3.0,4.0,3.0,3.0,2.0,4.0,2.0,1.0,1.0
installment_rate,4.0,2.0,2.0,3.0,4.0,1.0,1.0,2.0,4.0,1.0
personal_status_sex,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0
other_debtors,1.0,1.0,1.0,1.0,None,1.0,1.0,1.0,1.0,1.0


In [8]:
#SE BORRA LA COLUMNA DE MIXED
#Delete "mixed" Column
df.drop(columns=['mixed'], inplace=True) # se borra la columna que solo contiene basura

In [9]:
#MOSTRAMOS INFORMACION GENERAL DEL DATASET
#Show general information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   status                   1005 non-null   object
 1   duration                 1010 non-null   object
 2   credit_history           1003 non-null   object
 3   purpose                  1004 non-null   object
 4   amount                   1012 non-null   object
 5   savings                  1008 non-null   object
 6   employment_duration      1003 non-null   object
 7   installment_rate         1011 non-null   object
 8   personal_status_sex      1013 non-null   object
 9   other_debtors            1007 non-null   object
 10  present_residence        1005 non-null   object
 11  property                 1009 non-null   object
 12  age                      1009 non-null   object
 13  other_installment_plans  1010 non-null   object
 14  housing                  1015 non-null  

In [10]:
#DEFINIMOS LOS TIPOS DE VARIABLES
# Define variable types and convert data accordingly
# This section classifies the dataset columns into numerical, ordinal, and categorical/binary groups,
# then applies appropriate data type conversions for consistent analysis

# Variables numéricas:
num_col = ['duration', 'amount', 'age']

# Variables ordinales:
ord_col = ['employment_duration','installment_rate', 'present_residence','property', 'number_credits', 'job']

# Variables nominales & binarias:
cat_col = ['status', 'credit_history', 'purpose',  'savings','people_liable',
                   'personal_status_sex', 'other_debtors', 'other_installment_plans',
                   'housing', 'telephone', 'foreign_worker']

print("Para los datos de entrada, veamos la cantidad de cada tipo de variable obtenida:")
print("Variables numéricas:", len(num_col))
print("Variables ordinales:", len(ord_col))
print("Variables nominales & binarias:", len(cat_col))

df[num_col] = df[num_col].apply(pd.to_numeric, errors="coerce")
df[ord_col] = df[ord_col].astype("object")
df[cat_col] = df[cat_col].astype("object")

Para los datos de entrada, veamos la cantidad de cada tipo de variable obtenida:
Variables numéricas: 3
Variables ordinales: 6
Variables nominales & binarias: 11


In [11]:
#MOSTRAMOS LOS CAMBIOS
#Show general information of the dataset
#This helps verify that the previous corrections were successfully applied
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   status                   1005 non-null   object 
 1   duration                 1009 non-null   float64
 2   credit_history           1003 non-null   object 
 3   purpose                  1004 non-null   object 
 4   amount                   1008 non-null   float64
 5   savings                  1008 non-null   object 
 6   employment_duration      1003 non-null   object 
 7   installment_rate         1011 non-null   object 
 8   personal_status_sex      1013 non-null   object 
 9   other_debtors            1007 non-null   object 
 10  present_residence        1005 non-null   object 
 11  property                 1009 non-null   object 
 12  age                      1008 non-null   float64
 13  other_installment_plans  1010 non-null   object 
 14  housing                 

In [12]:
#MOSTRAMOS LA SUMA DE VALORES UNICOS
# Display the count of distinct values for each column (sorted from highest to lowest)
df.nunique().sort_values(ascending=False)

,0
amount,915
age,61
duration,41
purpose,30
credit_history,29
job,26
number_credits,24
status,23
savings,21
other_debtors,21


In [13]:
#GENERAMOS UNA DESCRIPCION ESTADISTICA DE LAS VARIABLES NUMERICAS
# Generate descriptive statistics for all numeric columns
df.select_dtypes(include="number").describe().T

,count,mean,std,min,25%,50%,75%,max
duration,1009.0,84.906838,1832.213663,4.0,12.00,18.0,24.00,58140.0
amount,1008.0,4173.232143,28013.363840,250.0,1371.75,2319.5,3976.75,887992.0
age,1008.0,45.996032,162.527743,19.0,27.00,33.0,42.00,4606.0


In [14]:
#MOSTRAMOS LOS VALORES UNICOS DE LAS VARIABLE NUMERICAS
# Display frequency counts for each numeric column (including NaN), sorted by value
# This helps identify inconsistencies or potential data entry errors
for col in df.select_dtypes(include="number").columns:
    vc = df[col].value_counts(dropna=False)
    print(f"== {col} (by value ascending) ==")
    print(vc.sort_index())
    print("-" * 50)


== duration (by value ascending) ==
duration
4.0          6
5.0          1
6.0         74
7.0          5
8.0          7
9.0         50
10.0        28
11.0         9
12.0       179
13.0         4
14.0         4
15.0        64
16.0         2
18.0       114
20.0         8
21.0        30
22.0         2
24.0       180
26.0         1
27.0        13
28.0         3
30.0        39
33.0         3
36.0        83
39.0         6
40.0         1
42.0        12
45.0         5
47.0         1
48.0        51
54.0         2
60.0        13
72.0         1
248.0        1
416.0        1
570.0        1
792.0        1
840.0        1
882.0        1
2760.0       1
58140.0      1
NaN         11
Name: count, dtype: int64
--------------------------------------------------
== amount (by value ascending) ==
amount
250.0        1
276.0        1
338.0        1
339.0        1
343.0        1
            ..
15945.0      1
18424.0      1
20076.0      1
887992.0     1
NaN         12
Name: count, Length: 916, dtype: int64
---

In [15]:
#HACEMOS LAS CORRECCIONES
# Correct column value errors identified in the previous analysis
df["duration"] = df["duration"].astype("Float64").mask(df["duration"] > 72, other=pd.NA)
df["age"] = df["age"].astype("Float64").mask(df["age"] > 75, other=pd.NA)
df["amount"] = df["amount"].astype("Float64").mask(df["amount"] > 25000, other=pd.NA)

In [16]:
#VISUALIZAMOS LOS CAMBIOS
# Display frequency counts for each numeric column (including NaN), sorted by value
# This helps verify that the previous corrections were successfully applied
for col in df.select_dtypes(include="number").columns:
    vc = df[col].value_counts(dropna=False)
    print(f"== {col} (by value ascending) ==")
    print(vc.sort_index())
    print("-" * 50)

== duration (by value ascending) ==
duration
4.0       6
5.0       1
6.0      74
7.0       5
8.0       7
9.0      50
10.0     28
11.0      9
12.0    179
13.0      4
14.0      4
15.0     64
16.0      2
18.0    114
20.0      8
21.0     30
22.0      2
24.0    180
26.0      1
27.0     13
28.0      3
30.0     39
33.0      3
36.0     83
39.0      6
40.0      1
42.0     12
45.0      5
47.0      1
48.0     51
54.0      2
60.0     13
72.0      1
<NA>     19
Name: count, dtype: Int64
--------------------------------------------------
== amount (by value ascending) ==
amount
250.0       1
276.0       1
338.0       1
339.0       1
343.0       1
           ..
15672.0     1
15945.0     1
18424.0     1
20076.0     1
<NA>       13
Name: count, Length: 915, dtype: Int64
--------------------------------------------------
== age (by value ascending) ==
age
19.0     2
20.0    14
21.0    14
22.0    27
23.0    45
24.0    45
25.0    40
26.0    51
27.0    50
28.0    44
29.0    36
30.0    40
31.0    37
32.0   

In [17]:
#MOSTRAMOS LOS VALORES UNICOS DE LAS VARIABLE OBJETO
# Display frequency counts for each non numeric columns (including NaN), sorted by value
# This helps identify inconsistencies or potential data entry errors
for col in df.columns:
    if df[col].dtype == "object":
        print(df[col].value_counts(dropna=False))
        print('-' * 50)

status
4.0        378
2.0        260
1.0        256
3.0         57
None        15
 4.0        12
 1.0        11
 2.0         8
 3.0         6
?            2
error        2
518.0        1
916.0        1
 360.0       1
497.0        1
64.0         1
444.0        1
96.0         1
76.0         1
92.0         1
685.0        1
144.0        1
104.0        1
667.0        1
Name: count, dtype: int64
--------------------------------------------------
credit_history
2.0       497
4.0       273
3.0        83
1.0        45
0.0        38
 2.0       22
None       17
 4.0       16
 3.0        5
 1.0        3
 0.0        2
172.0       1
100.0       1
907.0       1
166.0       1
 60.0       1
112.0       1
531.0       1
error       1
216.0       1
260.0       1
894.0       1
70.0        1
601.0       1
477.0       1
645.0       1
389.0       1
40.0        1
709.0       1
446.0       1
Name: count, dtype: int64
--------------------------------------------------
purpose
3.0        262
0.0        216
2.0   

In [18]:
#HACEMOS LAS CORRECCIONES
# Correct column value errors identified in the previous analysis
obj_cols = df.select_dtypes(include=["object", "string"]).columns
df[obj_cols] = df[obj_cols].replace(r"\s+", "", regex=True)
counts = df[obj_cols].apply(lambda s: s.map(s.value_counts(dropna=False)))
df[obj_cols] = df[obj_cols].mask(counts < 7)
df['credit_risk'] = pd.to_numeric(df['credit_risk'], errors='coerce')
df = df[df['credit_risk'].isin([0.0, 1.0])]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 992 entries, 0 to 1019
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   status                   961 non-null    object 
 1   duration                 973 non-null    Float64
 2   credit_history           959 non-null    object 
 3   purpose                  963 non-null    object 
 4   amount                   981 non-null    Float64
 5   savings                  969 non-null    object 
 6   employment_duration      966 non-null    object 
 7   installment_rate         976 non-null    object 
 8   personal_status_sex      976 non-null    object 
 9   other_debtors            965 non-null    object 
 10  present_residence        965 non-null    object 
 11  property                 969 non-null    object 
 12  age                      972 non-null    Float64
 13  other_installment_plans  968 non-null    object 
 14  housing                  975 n

In [20]:
# Save Clean DataFrame as Parquet
df.to_parquet('trabajo_grupal_mlops/data/02_df_data_preparation_01.parquet', index=False)

print("DataFrame saved successfully'")


DataFrame saved successfully'


In [21]:
stop_import_capture()

🛑 Import capture STOPPED

🔹 IMPORTED PACKAGES (requirements format)
Cython==3.0.12
bottleneck==1.4.2
charset_normalizer==3.4.4
cloudpickle==3.1.1
cycler==0.12.1
cython==3.0.12
dateutil==2.9.0.post0
docutils==0.21.2
ipython_genutils==0.2.0
ipywidgets==7.7.1
matplotlib==3.10.0
numexpr==2.14.1
numpy==2.0.2
packaging==25.0
pandas==2.2.2
patsy==1.0.2
pyarrow==18.1.0
pytz==2025.2
scipy==1.16.3
seaborn==0.13.2
statsmodels==0.14.5
zmq==26.2.1


In [22]:
# =============================================================
# System & Environment Information for Google Colab
# =============================================================
import sys
import subprocess

print("============================================")
print("🔹 PYTHON VERSION")
print("============================================")
!python --version

print("\n============================================")
print("🔹 PIP VERSION")
print("============================================")
!pip --version

🔹 PYTHON VERSION
Python 3.12.12

🔹 PIP VERSION
pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
